In [1]:
!pip install pykrx

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pykrx import stock
from pykrx import bond



In [3]:
import json

def get_market_ohlcv(start_date, end_date, ticker):
    stock_name = stock.get_market_ticker_name(ticker)
    df = stock.get_market_ohlcv(start_date, end_date, ticker)
    df['종목명'] = [stock_name] * len(df)

    return json.dumps(df.to_dict(orient='records'), ensure_ascii=False)

In [4]:
print(get_market_ohlcv("20220720", "20220810", "005930"))

[{"시가": 61800, "고가": 62100, "저가": 60500, "종가": 60500, "거래량": 16782238, "등락률": -0.6568144499178982, "종목명": "삼성전자"}, {"시가": 61100, "고가": 61900, "저가": 60700, "종가": 61800, "거래량": 12291374, "등락률": 2.1487603305785123, "종목명": "삼성전자"}, {"시가": 61800, "고가": 62200, "저가": 61200, "종가": 61300, "거래량": 10261310, "등락률": -0.8090614886731391, "종목명": "삼성전자"}, {"시가": 60900, "고가": 61900, "저가": 60800, "종가": 61100, "거래량": 9193681, "등락률": -0.3262642740619902, "종목명": "삼성전자"}, {"시가": 60800, "고가": 61900, "저가": 60800, "종가": 61700, "거래량": 6597211, "등락률": 0.9819967266775778, "종목명": "삼성전자"}, {"시가": 61300, "고가": 61900, "저가": 61200, "종가": 61800, "거래량": 7320997, "등락률": 0.1620745542949757, "종목명": "삼성전자"}, {"시가": 62300, "고가": 62600, "저가": 61600, "종가": 61900, "거래량": 10745302, "등락률": 0.16181229773462785, "종목명": "삼성전자"}, {"시가": 62400, "고가": 62600, "저가": 61300, "종가": 61400, "거래량": 15093120, "등락률": -0.8077544426494345, "종목명": "삼성전자"}, {"시가": 61000, "고가": 61700, "저가": 60300, "종가": 61300, "거래량": 13154816, "등락률": -0.1628664495114

In [7]:
tools = [
    {
        "type": "function",
        "function":
        {
            "name": "get_market_ohlcv",
            "description": "특정 종목에 대해 정해진 기간의 주가 데이터를 돌려줍니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "start_date": {
                        "type": "string",
                        "description": "YYYYMMDD 형식의 시작 날짜",
                    },
                    "end_date": {
                        "type": "string",
                        "description": "YYYYMMDD 형식의 종료 날짜",
                    },
                    "ticker": {
                        "type": "string",
                        "description": "6자리 숫자 형식의 종목 티커코드",
                    },
                },
                "required": ["start_date", "end_date", "ticker"],
            },
        }
    }
]


In [8]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

messages= [
  {
    "role": "user",
    "content": "삼성중공업(005930)의 2024년 12월부터 2025년 1월까지 주가를 분석해줘."
  }
]

response = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=messages,
  temperature=0,
  max_tokens=1024,
  tools=tools,
  tool_choice="auto" if tools is not None else None,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_4eipC4YOzqNWfcTTHCmVhUJU', function=Function(arguments='{"start_date": "20241201", "end_date": "20250131", "ticker": "005930"}', name='get_market_ohlcv'), type='function')])


In [9]:
available_functions = {
    "get_market_ohlcv": get_market_ohlcv,
}

for tool_call in response.choices[0].message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        start_date=function_args['start_date'],
        end_date=function_args['end_date'],
        ticker=function_args['ticker']
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "function",
            "name": function_name,
            "content": json.dumps(function_response, ensure_ascii=False)
        }
    )
    
second_response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
print(second_response.choices[0].message.content)

삼성중공업(005930)의 2024년 12월부터 2025년 1월까지의 주가 변동 분석을 다음과 같이 진행하겠습니다.

### 주가 분석

1. **시가, 고가, 저가, 종가**:
   - 기간 내 주가의 변동성을 볼 때, 시가와 고가, 저가, 종가가 긴 범위에서 변동하고 있음을 알 수 있습니다. 이는 주가가 불확실한 시장 상황에서 활성화되고 있음을 나타냅니다.

2. **변동성**:
   - 주가의 변동폭이 상대적으로 크며, 특정 날짜에 거래량이 급증한 경우가 있습니다. 이는 특정 뉴스나 이벤트가 영향을 미쳤을 가능성을 시사합니다.
   - 특히, 2025년 1월 초에 고가가 두드러지며 종가가 나타났고, 거래량도 비교적 많았습니다.

3. **거래량**:
   - 2024년 12월부터 2025년 1월까지의 거래량은 꾸준히 높은 편입니다. 이는 시장 참여자들이 많이 활동하고 있다는 것을 나타내며, 주가에 큰 영향을 미칠 수 있습니다.

4. **등락률**:
   - 등락률을 살펴보면, 상승과 하락을 반복적으로 경험하고 있습니다. 이를 통해 삼성중공업의 주가가 시장의 뉴스나 경제 지표에 민감하다는 것을 알 수 있습니다.
   - 특히, 일부 날에는 3% 이상의 큰 등락률을 보인 날도 있어, 이러한 날에는 외부적인 요인이나 내부 실적 발표가 있었던 것으로 추정됩니다.

### 결론
삼성중공업의 주가는 2024년 12월부터 2025년 1월까지의 기간 동안 활발한 거래가 이루어지면서 변동성이 있었습니다. 이러한 정보는 투자자들에게 중요한 신호가 될 수 있으며, 삼성중공업의 기본적인 펀더멘털뿐만 아니라 글로벌 경제 상황, 경쟁사 동향 등을 고려하여 최종 투자 결정을 내리는 것이 바람직합니다.
